# Solving SAT problem with Grover's algorithm

The **"Solving SAT problem with Grover's algorithm"** quantum kata is a series of exercises designed
to get you comfortable with using Grover's algorithm to solve realistic problems
using boolean satisfiability problems (SAT) as an example.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty; 
harder ones are marked with asterisks.

## Part I. Oracles for SAT problems

The most interesting part of learning Grover's algorithm is solving realistic problems.
This means using oracles which express an actual problem and not simply hard-code a known solution.
In this section we'll learn how to express boolean satisfiability problems as quantum oracles.

### Task 1.1. The AND oracle: $f(x) = x_0 \wedge x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if all qubits of the query register are in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [1]:
%kata T11_Oracle_And 

operation Oracle_And (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    //
    Controlled X(queryRegister, target);
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.1.-The-AND-oracle:-$f(x)-=-x_0-\wedge-x_1$).*

### Task 1.2. The OR oracle: $f(x) = x_0 \vee x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if at least one qubit of the query register is in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [2]:
%kata T12_Oracle_Or 

operation Oracle_Or (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    //
    X(target);
    ControlledOnInt(0, X)(queryRegister, target);
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.2.-The-OR-oracle:-$f(x)-=-x_0-\vee-x_1$).*

### Task 1.3. The XOR oracle: $f(x) = x_0 \oplus x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if the qubits of the query register are in different states, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [5]:
%kata T13_Oracle_Xor 

operation Oracle_Xor (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // Alternating qubits? No. Odd parity pattern.
    ApplyToEachCA(CNOT(_, target), queryRegister);
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.3.-The-XOR-oracle:-$f(x)-=-x_0-\oplus-x_1$).*

### Task 1.4. Alternating bits oracle: $f(x) = (x_0 \oplus x_1) \wedge (x_1 \oplus x_2) \wedge \dots \wedge (x_{N-2} \oplus x_{N-1})$

**Inputs:** 

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

> This oracle marks two states similar to the state explored in task 1.2 of the GroversAlgorithm kata: 
$|10101...\rangle$ and $|01010...\rangle$.  
It is possible (and quite straightforward) to implement this oracle based on this observation; 
however, for the purposes of learning to write oracles to solve SAT problems we recommend using the representation above.

<details>
  <summary><b>Need a hint? Click here</b></summary>
  Remember that you can use operations defined in previous tasks.
</details>

In [12]:
%kata T14_Oracle_AlternatingBits 

open Microsoft.Quantum.Arrays;

operation Oracle_AlternatingBits (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // Find the XOR of all pairs of consecutive qubits and store the result in queryRegister itself
    within {
        for i in 0 .. Length(queryRegister) - 2 {
            // Preserving order for each subsequent XOR check
            CNOT(queryRegister[i+1], queryRegister[i]);
        }
    }
    apply { 
        // For the eigenstates of interest, the intermediate states will have all 1s for Most
        Controlled X(Most(queryRegister), target); 
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.4.-Alternating-bits-oracle:-$f(x)-=-(x_0-\oplus-x_1)-\wedge-(x_1-\oplus-x_2)-\wedge-\dots-\wedge-(x_{N-2}-\oplus-x_{N-1})$).*

### Task 1.5. Evaluate one clause of a SAT formula

> For SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of several clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$clause(x) = \bigvee_k y_{k},\text{ where }y_{k} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$
>
> For example, one of the possible clauses on two variables is:
>
> $$clause(x) = x_0 \vee \neg x_1$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 1-dimensional array of tuples `clause` which describes one clause of a SAT problem instance $clause(x)$.

`clause` is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$).

**Example:**

* The clause $x_0 \vee \neg x_1$ can be represented as `[(0, true), (1, false)]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

In [22]:
%kata T15_Oracle_SATClause 

operation Oracle_SATClause (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    // 4N time complexity and cleaner vs 5N or 6N solution.
    // Given 1.4 solutions, using an auxiliary register is acceptable and generally practiced.
    let N = Length(clause);
    use aux = Qubit[N];
    within {
        for i in 0 .. N - 1 {
            let (qubitIndex, qubitState) = clause[i];
            let Oracle = ControlledOnInt(qubitState ? 1 | 0, X);
            Oracle(queryRegister[qubitIndex..qubitIndex], aux[i]);
        }
    }
    apply { Oracle_Or(aux, target); }
    //within {
    //    for (qubitIndex, qubitState) in clause {
    //        (qubitState ? I | X)(queryRegister[qubitIndex]);
    //    }
    //}
    //apply {
        // Get indices of all variables in clause.
    //    let indexList = Mapped(Fst<Int, Bool>, clause);
        // Get all qubits which are part of clause.
    //    let clauseQubits = Subarray(indexList, queryRegister);
        // Compute disjunction of variables in clause.
    //    Oracle_Or(clauseQubits, target);
    //}
}

Testing SAT clause instance x2...
Testing SAT clause instance x1 ∨ ¬x0...
Testing SAT clause instance x0 ∨ ¬x2 ∨ x1...
Testing SAT clause instance x4 ∨ ¬x3 ∨ x2 ∨ x0...
Testing SAT clause instance ¬x3 ∨ ¬x4 ∨ x2 ∨ x0 ∨ x1...
Testing SAT clause instance ¬x1 ∨ ¬x4 ∨ ¬x5 ∨ ¬x0 ∨ ¬x2 ∨ x3...
Testing SAT clause instance ¬x0 ∨ x2 ∨ x1 ∨ ¬x3...
Testing SAT clause instance x1 ∨ x0 ∨ x2...
Testing SAT clause instance x4 ∨ ¬x1 ∨ x3 ∨ ¬x0 ∨ ¬x2...
Testing SAT clause instance ¬x2 ∨ x0 ∨ ¬x1...


Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.5.-Evaluate-one-clause-of-a-SAT-formula).*

### Task 1.6. k-SAT problem oracle

> For k-SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of $M$ clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$f(x) = \bigwedge_i \big(\bigvee_k y_{ik} \big),\text{ where }y_{ik} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 2-dimensional array of tuples `problem` which describes the k-SAT problem instance $f(x)$.

$i$-th element of `problem` describes the $i$-th clause of $f(x)$; 
it is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$).

**Example:**

A more general case of the OR oracle for 3 variables $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

In [23]:
%kata T16_Oracle_SAT 

operation Oracle_SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    //
    let N = Length(problem);
    use aux = Qubit[N];

    within {
        for i in 0 .. N - 1 {
            Oracle_SATClause(queryRegister, aux[i], problem[i]);
        }
    }
    apply { Oracle_And(aux, target); }
}

Testing 2-SAT instance (2, (x0 ∨ x1))...
Testing 2-SAT instance (2, (x0 ∨ x1) ∧ (¬x1 ∨ ¬x0))...
Testing 2-SAT instance (3, (¬x1 ∨ ¬x2) ∧ (x0 ∨ x1) ∧ (¬x1 ∨ ¬x0) ∧ (x2 ∨ x1))...
Testing k-SAT instance (3, (¬x2) ∧ (¬x0 ∨ x1))...
Testing k-SAT instance (4, (¬x1) ∧ (x1 ∨ x0 ∨ x3) ∧ (¬x3))...
Testing k-SAT instance (5, (¬x4 ∨ x2 ∨ x0 ∨ x1) ∧ (¬x3 ∨ x0 ∨ x4) ∧ (x4 ∨ x1) ∧ (¬x3 ∨ x2 ∨ x0 ∨ ¬x4))...
Testing k-SAT instance (6, (¬x0 ∨ x3 ∨ ¬x2) ∧ (x5) ∧ (x4) ∧ (x4) ∧ (x3 ∨ ¬x5))...


Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.6.-k-SAT-problem-oracle).*

## Part II. Oracles for exactly-1 3-SAT problem

Exactly-1 3-SAT problem (also known as "one-in-three 3-SAT") is a variant of a general 3-SAT problem.
It has a structure similar to a 3-SAT problem, but each clause must have *exactly one* true literal, 
while in a normal 3-SAT problem each clause must have *at least one* true literal.

### Task 2.1. "Exactly one $|1\rangle$" oracle

**Inputs:** 

  1. 3 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if exactly one bit of $x$ is in the $|1\rangle$ state, and $0$ otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [28]:
function GenerateBitString(length : Int, index : Int) : Bool[] {
    //
    mutable bitString = [false, size = length];
    set bitString w/= index <- true;
    return bitString;
}

GenerateBitString

In [29]:
%kata T21_Oracle_Exactly1One 

operation Oracle_Exactly1One (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // Start by generating the bit strings
    let N = Length(queryRegister);
    for i in 0 .. N - 1 {
        ControlledOnBitString(GenerateBitString(N, i), X)(queryRegister, target);
        // ControlledOnInt(2^i, X)(queryRegister, target);
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-2.1.-"Exactly-one-$|1\rangle$"-oracle).*

### Task 2.2. "Exactly-1 3-SAT" oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 2-dimensional array of tuples `problem` which describes the 3-SAT problem instance $f(x)$.

`problem` describes the problem instance in the same format as in task 1.6;
each clause of the formula is guaranteed to have exactly 3 terms.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

**Example:**

An instance of the problem $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`,
and its solutions will be `(true, false, false)`, `(false, true, false)` and `(false, false, true)`, 
but none of the variable assignments in which more than one variable is true, which are solutions for the general SAT problem.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Can you reuse parts of the code in section 1?
</details>

In [38]:
operation Oracle_SATClauseExactly1One (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    //
    let N = Length(clause);
    use aux = Qubit[N];
    within {
        for i in 0 .. N - 1 {
            let (qubitIndex, qubitState) = clause[i];
            let Oracle = ControlledOnInt(qubitState ? 1 | 0, X);
            Oracle(queryRegister[qubitIndex..qubitIndex], aux[i]);
        }
    }
    apply { Oracle_Exactly1One(aux, target); }
}

Oracle_SATClauseExactly1One

In [39]:
%kata T22_Oracle_Exactly1SAT 

operation Oracle_Exactly1_3SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    //
    let N = Length(problem);
    use aux = Qubit[N];

    within {
        for i in 0 .. N - 1 {
            Oracle_SATClauseExactly1One(queryRegister, aux[i], problem[i]);
        }
    }
    apply { Oracle_And(aux, target); }
}

Testing exactly-1 3-SAT instance (2, (¬x0 ∨ ¬x1))...
Testing exactly-1 3-SAT instance (3, (x2 ∨ ¬x0 ∨ ¬x1) ∧ (¬x0 ∨ x1 ∨ ¬x2))...
Testing exactly-1 3-SAT instance (4, (x2 ∨ ¬x1 ∨ x0) ∧ (x0 ∨ ¬x3 ∨ ¬x2) ∧ (¬x0 ∨ x2 ∨ ¬x1))...
Testing exactly-1 3-SAT instance (5, (¬x0 ∨ x2 ∨ x4) ∧ (x1 ∨ ¬x4 ∨ ¬x2) ∧ (x3 ∨ ¬x1 ∨ x0) ∧ (¬x1 ∨ ¬x3 ∨ ¬x2))...
Testing exactly-1 3-SAT instance (6, (x5 ∨ ¬x4 ∨ ¬x3) ∧ (x4 ∨ x5 ∨ x1) ∧ (¬x2 ∨ x3 ∨ x0) ∧ (¬x3 ∨ ¬x2 ∨ x0) ∧ (¬x5 ∨ ¬x4 ∨ ¬x3))...


Success!

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-2.2.-"Exactly-1-3-SAT"-oracle).*

## Part III. Using Grover's algorithm for problems with multiple solutions

### Task 3.1. Using Grover's algorithm

**Goal:**

Implement Grover's algorithm and use it to find solutions to SAT instances from parts I and II. 

> If you want to learn the Grover's algorithm itself, try doing [GroversAlgorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) first.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_GroversSearch_Algorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_GroversSearch_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<details>
  <summary><b>Need a hint? Click here</b></summary>
Experiment with SAT instances with different number of solutions and the number of algorithm iterations 
to see how the probability of the algorithm finding the correct answer changes depending on these two factors.

For example, 
* the AND oracle from task 1.1 has exactly one solution,
* the alternating bits oracle from task 1.4 has exactly two solutions,
* the OR oracle from task 1.2 for 2 qubits has exactly 3 solutions, and so on.
</details>

In [40]:
operation PhaseOracle (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {
    // 
    use q = Qubit();
    within {
        X(q);
        H(q);
    } 
    apply { markingOracle(register, q); }
}

PhaseOracle

In [41]:
function OracleConverter (markingOracle : ((Qubit[], Qubit) => Unit is Adj)) : (Qubit[] => Unit is Adj) {
    // marking oracle on input with ancilla = phase oracle on input
    return PhaseOracle(markingOracle, _);
}

OracleConverter

In [42]:
operation HadamardTransform (register : Qubit[]) : Unit is Adj {
    //
    ApplyToEachCA(H, register);
}

HadamardTransform

In [44]:
operation Oracle_ArbitraryPattern (queryRegister : Qubit[], target : Qubit, pattern : Bool[]) : Unit is Adj + Ctl {
    //
    ControlledOnBitString(pattern, X)(queryRegister, target);
}

Oracle_ArbitraryPattern

In [45]:
open Microsoft.Quantum.Math;

operation ConditionalPhaseFlip (register : Qubit[]) : Unit is Adj {
    // Controlled marking oracle
    let ControlledMarkingOracle = Oracle_ArbitraryPattern(_, _, [false, size = Length(register)]);
    let ControlledPhaseOracle = OracleConverter(ControlledMarkingOracle);
    // Controlled phase kickback
    ControlledPhaseOracle(register);
    // Applying a global phase flip, even though it's unphysical(?)
    // It doesn't matter for Grover's search algorithm itself, since the global phase is not observable, 
    // but can have side effects when used as part of other algorithms.
    R(PauliI, 2.0 * PI(), Head(register));
}

ConditionalPhaseFlip

In [46]:
operation GroverIteration (register : Qubit[], oracle : (Qubit[] => Unit is Adj)) : Unit is Adj {
    //
    oracle(register);
    within { HadamardTransform(register); }
    apply { ConditionalPhaseFlip(register); }
}

GroverIteration

In [47]:
operation GroversSearch (register : Qubit[], oracle : ((Qubit[], Qubit) => Unit is Adj), iterations : Int) : Unit {
    //
    ApplyToEachCA(H, register);
    RepeatA(GroverIteration, iterations, (register, OracleConverter(oracle)));
}

GroversSearch

In [59]:
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Diagnostics;

operation Run_GroversSearch_Algorithm () : Unit {
    //
    use qs = Qubit[4];
    
    Message("Running Grover's Search algorithm on 4-qubit AND oracle...");
    GroversSearch(qs, Oracle_And, 3);
    let results = ResultArrayAsInt(MultiM(qs)) == 15;
    Fact(results, "Oracle_And: A qubit was found in the 0 eigenstate.");
    ResetAll(qs);

    Message("Running Grover's Search algorithm on 4-qubit OR oracle...");
    GroversSearch(qs, Oracle_Or, 1);
    let results1 = ResultArrayAsInt(MultiM(qs)) > 0;
    Fact(results1, "Oracle_Or: All qubits in 0 eigenstate.");
    ResetAll(qs);

    Message("Running Grover's Search algorithm on 4-qubit AlternatingBits oracle...");
    GroversSearch(qs, Oracle_AlternatingBits, 2);
    let results2 = MultiM(qs);
    mutable previousResult = Zero;
    for i in 0 .. Length(qs) - 1 {
        if (i == 0) {
            set previousResult = results2[i];
        }
        elif (previousResult == Zero) { 
            EqualityFactR(results2[i], One, "Oracle_AlternatingBits: Qubit not in 1 state."); 
            set previousResult = results2[i];
        }
        else { 
            EqualityFactR(results2[i], Zero, "Oracle_AlternatingBits: Qubit not in 0 state."); 
            set previousResult = results2[i];
        }
    }
    ResetAll(qs);
}

Run_GroversSearch_Algorithm

In [61]:
%simulate Run_GroversSearch_Algorithm

Running Grover's Search algorithm on 4-qubit AND oracle...
Running Grover's Search algorithm on 4-qubit OR oracle...
Running Grover's Search algorithm on 4-qubit AlternatingBits oracle...


()

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-3.1.-Using-Grover's-algorithm).*

### Task 3.2. Universal implementation of Grover's algorithm

**Inputs:**

  1. The number of qubits N.

  2. A marking oracle which implements a boolean expression, similar to the oracles from part I.

**Output:**

An array of N boolean values which satisfy the expression implemented by the oracle 
(i.e., any basis state marked by the oracle).

> Note that the similar task in the [GroversAlgorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) required you to implement Grover's algorithm 
in a way that would be robust to accidental failures, but you knew the optimal number of iterations 
(the number that minimized the probability of such failure).
> 
> In this task you also need to make your implementation robust to not knowing the optimal number of iterations.
> You can see an example of doing that in [Exploring Grover's Algorithm tutorial](./../tutorials/ExploringGroversAlgorithm/ExploringGroversAlgorithm.ipynb).

In [ ]:
%kata T32_UniversalGroversAlgorithm 

operation UniversalGroversAlgorithm (N : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Bool[] {
    // ...
    return [false, size = N];
}

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-3.2.-Universal-implementation-of-Grover's-algorithm).*